# Clusterung von Bildern mit dem K-Means-Algorithmus
## Wichtig: hier müssen die Clusterpunkte als Integer definiert sein
 

Quelle:[Lehrtext der ETH-Zürich](http://www.educ.ethz.ch/unterrichtsmaterialien/mathematik/k-means-algorithmus.html)
Hier ein Beispiel:

<table><tr><td><img src='cover_kl.png'></td><td><img src='cover_ausgabe_kl.png'></td></tr>
<tr><td>Ausgangsbild</td><td>Bild komprimiert mit nur 5 Farben</td></tr>
</table>




In [12]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import random
class k_Means():
    """
        Erhaelt eine Punkt-Liste als Array
        die Anzahl der Clusterpunkte
        eventuell die Cluster-Liste, wenn nicht,
        werden die Cluster-Punkte zufaellig aus den
        Punkten der PL bestimmt
    """
    def __init__(self, punkte, anzahl, cetroiden=None):        
        #print(self.cetroiden)
        self.puLi = punkte
        self.anzahlCluster = anzahl
        self.laengePuLi, self.dimPunkte =self.puLi.shape
        if cetroiden:
            self.cetroiden=np.array(cetroiden)
        else:
            self.bestimmeCentroiden()
        #print("StartCluster: \n", self.cetroiden)
        self.cluster= np.zeros((self.laengePuLi,),dtype=int)
        self.berechneFehler()
        self.error=20
        self.cls=[]#Liste der Cetroiden 
        c=np.copy(self.cetroiden)#als echte Kopie einfuegen
        self.cls.append(c)#das sind die ersten
        
        
    def bestimmeCentroiden(self):
        #zufaellige Auswahl der Clusterpunkte aus den gegebenen
        self.cetroiden=np.zeros((self.anzahlCluster,self.dimPunkte),dtype=int)
        cetroidenste=[]
        i=0
        while len(cetroidenste) < self.anzahlCluster:
            index= random.randint(0,self.laengePuLi-1)
            cl=self.puLi[index]
            if not self.istEnthalten(cetroidenste,cl): #vermeide Doppelte
                cetroidenste.append(cl)
                self.cetroiden[i]=self.puLi[index]
                i+=1
                
    def istEnthalten(self,array,el):
        for e in array:
            if (e==el).all():
                return True
        return False
        
    def einteilRunde(self):   
        """
        ordnet die Punkte den Centroiden zu
        """
        for i in range(self.laengePuLi):
            #print("Abstaende: ",self.puLi[i])
            abstand=np.linalg.norm(self.cetroiden-self.puLi[i],axis=1)
            #euklid. Abstand zur Centroide       
            self.cluster[i]=np.argmin(abstand) #Bestimme Index des Minimums
        #print("ClusterListe: ",self.cluster)
        self.cetroiden_old=np.copy(self.cetroiden)        
        for i in range(self.anzahlCluster):            
            punkte= [self.puLi[j] for j in range(self.laengePuLi) if self.cluster[j]==i]                 
            self.cetroiden[i]=np.mean(punkte,axis =0)        
                
        c=np.copy(self.cetroiden)
        self.cls.append(c)        
        self.diff = np.linalg.norm(self.cetroiden-self.cetroiden_old)
        
        self.berechneFehler()
        

    def berechneFehler(self):
        """
        bestimmt jeweils den euklidischen Abstand der Centroiden
        zu den einzelnen Clusterpunkten
        """
        error=0
        for i in range(self.anzahlCluster): 
            diff = np.array([self.puLi[j] -self.cetroiden[i] for j in range(self.laengePuLi) if self.cluster[j]==i])
            d= np.linalg.norm(diff)
            #print(diff,d)
            error+=d
        #print("Fehler: ",error)


    
    def einteilen(self,bis=0): 
        """
        bis legt fest, wie groß der Abstand der Centroiden 
        zu denen der letzten Runde ist
        """
        
        self.einteilRunde()
        while self.diff> bis:    
            self.einteilRunde()  
        #self.zeigen()
            
            
    def gibZahlen(self):
        print("Verteilung der Punkte auf die Cluster")
        #liefert die Anzahlen der Punkte je Cluster
        for i in range(self.anzahlCluster):            
            punkte= [self.puLi[j] for j in range(self.laengePuLi) if self.cluster[j]==i]
            print(i,len(punkte))

    def zeigeCentroiden(self):
        print("Centroiden")
        for c in self.cls:
            print(c)
    """            
    def zeigen(self):
        fig = plt.figure()
        fig.suptitle("Einteilung in Cluster")
        ax = fig.add_subplot(111)        
        farben=["red","yellow","blue","green","lightblue","grey"]*2
        m=["+","x","*","o","v"]*20
        for i in range(self.anzahlCluster):
            xs= [self.puLi[j][0] for j in range(self.laengePuLi) if self.cluster[j]==i]
            ys= [self.puLi[j][1] for j in range(self.laengePuLi) if self.cluster[j]==i]                      
            ax.scatter(xs,ys, c=farben[i])        
        si=0#Markerzaehler
        self.zeigeCentroiden()        
        for c in self.cls:            
            ax.scatter(c[0][0],c[0][1], c="black",marker=m[si],label=si)#damit das Label nur einmal kommt
            for i in range(1,self.anzahlCluster):
                ax.scatter(c[i][0],c[i][1], c="black",marker=m[si])#der Rest ohne Label
            si+=1

        maxis=np.amax(self.puLi,0)
        minis=np.amin(self.puLi,0)
        #print("Maxis ",maxis, " Minis: ",minis)
        plt.axis([int(minis[0])-2,int(maxis[0])+2,
                  int(minis[1])-2,int(maxis[1])+2])
        plt.legend()
        plt.show()
    """

In [15]:
class BildKMeans:
    def __init__(self,datei,anz):
        """
        bekommt den Dateinamen des Bildes und die Anzahl der Farben
        """
        self.datei=datei
        self.anz=anz
        
    def clustern(self,bis=0):
        #wie groß darf die Abweichung sein, default=0
        bk.ppm2array()
        self.k=k_Means(bk.piArray,self.anz)
        self.k.einteilen(bis)        

    def ppm2array(self):
        #einlesen und Array erstellen
        f = open (datei,"r")
        inhalt= f.readlines()
        self.kopf=inhalt[:4]
        groesse=self.kopf[2].strip().split()
        
        self.hoehe=int(groesse[1])
        self.breite=int(groesse[0])
        print(groesse, self.breite, self.hoehe)
        bild=inhalt[4:]
        bli=[]
        for z in bild:
            pl = z.strip().split(" ")
            bli+=pl
        pili=[]
        for i  in range(0,len( bli),3):
            r= int(bli[i])
            g= int(bli[i+1])
            b= int(bli[i+2])
            pili.append([r,g,b])
        self.piArray=np.array(pili) #Startwerte des Bildes speichern
        #print(self.piArray, self.piArray.shape)
        #print(self.kopf)
        
    def array2ppm(self):
        #schreibe zurueck in Datei mit richtigem Format
        kopf=''
        for k in self.kopf:
            kopf+=k
        kopf=kopf[:-1]
        print(kopf)
        ausgabe=self.datei[:-4]+"_ausgabe.ppm"
        bAr=np.copy(self.k.puLi)        
        for i in range(self.k.laengePuLi):
            bAr[i]=self.k.cetroiden[self.k.cluster[i]]
        bAr=bAr.reshape(self.breite*3,self.hoehe)
        #bAr=np.array2string(bAr)
        print(bAr,bAr.shape)
        np.savetxt(ausgabe, bAr, fmt="%d" ,
                   delimiter=" ",header=kopf,
                   comments='')
        
        

In [16]:
datei="cover.ppm"
bk=BildKMeans(datei,5)
bk.clustern(10)
bk.array2ppm()

['600', '600'] 600 600
P3
# Created by IrfanView
600 600
255
[[  5  72 210 ...   5  72 210]
 [  5  72 210 ...   5  72 210]
 [  5  72 210 ...   5  72 210]
 ...
 [  8  29 159 ...   5  72 210]
 [  5  72 210 ...   5  72 210]
 [  5  72 210 ...   5  72 210]] (1800, 600)
